In [5]:
import cvxpy as cp
import numpy as np
import pandas as pd
import plotly.express as ex

In [105]:
def create_toy_covariance_matrix():
    cor = [[1, 0.6, 0.5],
           [0.6, 1, 0.4],
           [0.5, 0.4, 1]]
    cor = np.array(cor)
    # monthly vol
    vol = np.array([0.2, 0.3, 0.6])*np.sqrt(1/12)
    vol = np.diag(vol)
    var = vol.dot(cor).dot(vol)
    return var

def create_toy_returns():
    ret = np.array([0.2, 0.3, 0.6])*0.3*np.sqrt(1/12)
    return ret

var = create_toy_covariance_matrix()
ret = create_toy_returns()
n = var.shape[0]

In [106]:
lda = 1
def sharpe_optimization(lda, ta=20/1e04):
    n = var.shape[0]
    w = cp.Variable(n)
    objfun = (1/2)*cp.quad_form(w, var) - lda* ret.T @ w +\
             ta*cp.norm(w, 1)
    constraints = [w >= 0, sum(w) == 1]
    prob = cp.Problem(cp.Minimize(objfun), constraints)
    prob.solve()
    pvol = np.sqrt(w.value.T.dot(var).dot(w.value))
    pret = w.value.T.dot(ret) - ta
    return pd.DataFrame([pvol, pret, lda, ta] + w.value.tolist()).T

df  = [sharpe_optimization(x) for x in np.linspace(0, 3, num=50)]
df += [sharpe_optimization(x, 0) for x in np.linspace(0, 3, num=50)]
df = pd.concat(df)
df.columns = ['vol', 'return', 'lda', 'stamp', 'w0', 'w1', 'w2']

In [107]:
# results for linear cost objective
ex.line(df, x='vol', y='return', hover_name='lda',
           hover_data=['w0', 'w1', 'w2'], color='stamp')

# Simple impact costs
Assuming our transaction costs follow the square root model

\begin{align}
r_i = \sigma_i \sqrt{\text{DTC}_i} \\
\text{DTC}_i = \dfrac{\text{Notional}\  w_i}{ V_i} \ ,
\end{align}
we can add this term as a negative return in the objective function:
\begin{align}
\sum w_i r_i = \sum_i \sigma_i \sqrt{\dfrac{\text{Notional} }{ V_i}} w_i^{3/2}
\end{align}

In [111]:
lda = 1
traded_value = np.array([2, 5, 10])
def impact_optimization(notional, ta=20/1e04):
    n = var.shape[0]
    w = cp.Variable(n)
    objfun = (1/2)*cp.quad_form(w, var) - ret.T @ w +\
             ta*cp.norm(w, 1)
    dvol = np.sqrt(np.diag(var))*1/np.sqrt(21)
    ifactor = np.sqrt(notional)/np.sqrt(traded_value)*dvol
    impact_term = cp.sum( cp.multiply(ifactor,w**(3/2)))
    objfun += impact_term
    constraints = [w >= 0, sum(w) == 1]
    prob = cp.Problem(cp.Minimize(objfun), constraints)
    prob.solve()
    pvol = np.sqrt(w.value.T.dot(var).dot(w.value))
    pret = w.value.T.dot(ret) - ta - impact_term.value
    return pd.DataFrame([pvol, pret, lda, ta, notional] +\
                        w.value.tolist()).T

df = [impact_optimization(notional=x) for x in np.linspace(1, 5, num=50)]
df = pd.concat(df)
df.columns = ['vol', 'return', 'lda', 'stamp', 'notional', 'w0', 'w1', 'w2']

In [114]:
# results for linear cost objective
ex.line(df, x='notional', y='return', hover_name='vol',
           hover_data=['w0', 'w1', 'w2'])